In [55]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\sudha\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [56]:
pip install geocode

  Using cached geocode-0.1.0.tar.gz (2.0 kB)
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'c:\Users\sudha\AppData\Local\Programs\Python\Python39\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\sudha\\AppData\\Local\\Temp\\pip-install-o1cetkmo\\geocode_0c01bfcf9d6c460cb8ab14fa9f51de9e\\setup.py'"'"'; __file__='"'"'C:\\Users\\sudha\\AppData\\Local\\Temp\\pip-install-o1cetkmo\\geocode_0c01bfcf9d6c460cb8ab14fa9f51de9e\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\sudha\AppData\Local\Temp\pip-pip-egg-info-8j36m7gs'
         cwd: C:\Users\sudha\AppData\Local\Temp\pip-install-o1cetkmo\geocode_0c01bfcf9d6c460cb8ab14fa9f51de9e\
    Complete output (10 lines):
    Traceback (most recent call last):
      File "<string>", line 1

In [57]:
import os
import cv2
import torch
import geocoder
from geopy.geocoders import Nominatim
from datetime import datetime
from twilio.rest import Client

In [58]:

# Load YOLOv5 model (adjust path if using a custom trained model)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-24 Python-3.9.9 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [59]:
# Twilio credentials
account_sid = "AC0fe931584d037885efa617d4e98ec3c4"
auth_token = "1c50debfa2cad6427725b359370ffd48"
client = Client(account_sid, auth_token)

In [60]:
def detect_accident(video_path):
    cap = cv2.VideoCapture(video_path)
    count = 0
    accident_detected = False
    accident_details = ""
    vehicle_category = "Unknown"
    accident_type = "Collision"

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform YOLOv5 object detection
        results = model(frame)
        detected_objects = results.pandas().xyxy[0]["name"].tolist()

        # Check for vehicles and potential collision detection logic
        if "car" in detected_objects or "truck" in detected_objects or "bus" in detected_objects:
            vehicle_category = "Car/Truck/Bus"
            
            # Placeholder logic to simulate accident detection
            if count % 100 == 0:  # Replace with actual collision detection logic
                accident_detected = True
                accident_details = f"Accident detected at frame {count}\n"
        
        count += 1

    cap.release()
    return accident_detected, accident_details, vehicle_category, accident_type

In [61]:

def get_location():
    try:
        g = geocoder.ip('me')
        geoLoc = Nominatim(user_agent="GetLoc")
        locname = geoLoc.reverse(g.latlng)
        return locname.address if locname else "Unknown Location"
    except Exception as e:
        print(f"Location Error: {e}")
        return "Unknown Location"

In [62]:


def save_accident_report(details, location, vehicle_category, accident_type):
    accident_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open("Accident.txt", "a") as file:  # Append instead of overwrite
            file.write(f"\n--- Accident Report ---\n")
            file.write(f"Date: {accident_date}\n")
            file.write(f"Vehicle Category: {vehicle_category}\n")
            file.write(f"Accident Type: {accident_type}\n")
            file.write(f"{details}\nLocation: {location}\n")
    except Exception as e:
        print(f"File Writing Error: {e}")

def send_alert(location, vehicle_category, accident_type):
    accident_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message_body = f"🚨 Accident detected!\nDate: {accident_date}\nVehicle: {vehicle_category}\nType: {accident_type}\nLocation: {location}. Check Accident.txt for details."
    
    try:
        client.messages.create(
            body=message_body,
            from_="18723269956",
            to="919981302302"
        )
    except Exception as e:
        print(f"Twilio Error: {e}")

def process_video(video_path):
    location = get_location()
    accident_occurred, details, vehicle_category, accident_type = detect_accident(video_path)

    if accident_occurred:
        save_accident_report(details, location, vehicle_category, accident_type)
        send_alert(location, vehicle_category, accident_type)
        return f"🚨 Accident detected!\nDetails: {details}\nLocation: {location}"
    else:
        return "✅ No accident detected."




In [63]:
# Example Usage
video_path = "accident.mp4"
result = process_video(video_path)
print(result)

C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models

Twilio Error: HTTP 429 error: Unable to create record: Account AC0fe931584d037885efa617d4e98ec3c4 exceeded the null daily messages limit
🚨 Accident detected!
Details: Accident detected at frame 200

Location: Indore City, Indore, Juni Indore Tahsil, Indore, Madhya Pradesh, 452001, India


In [1]:
import os
import cv2
import torch
import geocoder
from geopy.geocoders import Nominatim
from datetime import datetime
from twilio.rest import Client

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-24 Python-3.9.9 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
account_sid = "AC0fe931584d037885efa617d4e98ec3c4"
auth_token = "1c50debfa2cad6427725b359370ffd48"
client = Client(account_sid, auth_token)


In [8]:
def detect_accident(video_path):
    cap = cv2.VideoCapture(video_path)
    accident_detected = False
    accident_details = ""
    bounding_boxes = []
    frame_count = 0
    accident_frame = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1

        
        results = model(frame)
        df = results.pandas().xyxy[0]  

        # Filter detected vehicles
        detected_vehicles = df[(df['name'].isin(["car", "truck", "bus"])) & (df['confidence'] > 0.50)]

        # Draw Bounding Boxes
        for _, row in detected_vehicles.iterrows():
            xmin, ymin, xmax, ymax = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])
            label = f"{row['name']} ({row['confidence']:.2f})"
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(frame, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            bounding_boxes.append(f"{label} at ({xmin}, {ymin}) to ({xmax}, {ymax})")

        
        if len(detected_vehicles) > 1 and frame_count % 100 == 0:
            accident_detected = True
            accident_details = f"Accident detected at frame {frame_count}"
            accident_frame = frame.copy()  # Save accident frame

        if accident_detected:
            break  

    cap.release()


    if accident_detected and accident_frame is not None:
        cv2.imwrite("accident_frame.jpg", accident_frame)

    return accident_detected, accident_details, bounding_boxes

def get_location():
    """ Get the location using geolocation services. """
    try:
        g = geocoder.ip('me')
        geoLoc = Nominatim(user_agent="GetLoc")
        locname = geoLoc.reverse(g.latlng)
        return locname.address if locname else "Unknown Location"
    except Exception as e:
        print(f"Location Error: {e}")
        return "Unknown Location"

def save_accident_report(details, location, bounding_boxes):
    """ Save accident report details to Accident.txt """
    accident_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open("Accident.txt", "a") as file:
            file.write(f"\n--- Accident Report ---\n")
            file.write(f"Date: {accident_date}\n")
            file.write(f"Accident Details: {details}\n")
            file.write(f"Location: {location}\n")
            file.write("Bounding Boxes:\n")
            for bbox in bounding_boxes:
                file.write(f"  - {bbox}\n")
    except Exception as e:
        print(f"File Writing Error: {e}")

def send_alert(location, bounding_boxes):
    """ Send an alert message via Twilio. """
    accident_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    bbox_summary = "\n".join(bounding_boxes[:3])  # Only include first 3 for SMS

    message_body = (
        f"🚨 Accident Alert!\nDate: {accident_date}\n"
        f"Location: {location}\n"
        f"Details:\n{bbox_summary}\nCheck Accident.txt for full report."
    )

    try:
        message = client.messages.create(
            body=message_body,
            from_="18723269956",
            to="919981302302"
        )
        print(f"🚀 SMS Sent! SID: {message.sid}")
    except Exception as e:
        print(f"Twilio Error: {e}")

def process_video(video_path):
    """ Process video and detect accident events. """
    location = get_location()
    accident_occurred, details, bounding_boxes = detect_accident(video_path)

    if accident_occurred:
        save_accident_report(details, location, bounding_boxes)
        send_alert(location, bounding_boxes)
        return f"🚨 Accident detected!\nDetails: {details}\nLocation: {location}\nBounding Boxes:\n" + "\n".join(bounding_boxes)
    else:
        return "✅ No accident detected."


In [9]:
video_path = "accident.mp4"
result = process_video(video_path)
print(result)

Location Error: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=22.7179&lon=75.8333&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D6350E74C0>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 11001] getaddrinfo failed)"))


C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\sudha/.cache\torch\hub\ultralytics_yolov5_master\models

Twilio Error: HTTPSConnectionPool(host='api.twilio.com', port=443): Max retries exceeded with url: /2010-04-01/Accounts/AC0fe931584d037885efa617d4e98ec3c4/Messages.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D62E50ADC0>: Failed to resolve 'api.twilio.com' ([Errno 11001] getaddrinfo failed)"))
🚨 Accident detected!
Details: Accident detected at frame 200
Location: Unknown Location
Bounding Boxes:
car (0.89) at (1, 0) to (226, 113)
car (0.89) at (0, 0) to (229, 115)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (1, 0) to (226, 113)
car (0.88) at (0, 0) to (230, 116)
car (0.88) at (0, 0) to (230, 116)
car (0.88) at (1, 0) to (226, 115)
car (0.88) at (0, 0) to (230, 116)
car (0.88) at (0, 0) to (231, 116)
car (0.89) at (0, 0) to (233, 117)
car (0.89) at (0, 0) to (233, 117)
car (0.90) at (0, 0) to (234, 